# Housing Rental Analysis for San Francisco

This analysis considers data from the San Francisco housing market and visualizes trends over time.
Using neighborhood-specific location data, an interactive GeoViews map is created.

In [32]:
import pandas as pd
import hvPlot.pandas
import geoviews
from pathlib import Path

#### Read CSV Data

In [33]:
sfo_data_df = pd.read_csv(
    Path("Resources/sfo_neighborhoods_census_data.csv",
         index_col="date",
         parse_dates=True,
         infer_datetime_format=True)
)

---

## Average Number of Housing Units

In [40]:
housing_units_by_year = sfo_data_df.groupby("year").mean()

housing_units_by_year.hvplot.bar(x="year",y="housing_units",
                                 yformatter="%f",
                                 rot=45,
                                 ylim=[360000,390000],
                                 ylabel="Housing Units",xlabel="Year",
                                 title="Housing Units")

:Bars   [year]   (housing_units)

---

## Average Sale Prices per Square Foot

In [41]:
prices_square_foot_by_year = sfo_data_df.groupby("year").mean().drop(columns=["housing_units"])
prices_square_foot_by_year.hvplot.line(yformatter="%f",
                                       ylabel="$ Amount",xlabel="Year",
                                       title="Price per Square Foot")


:NdOverlay   [Variable]
   :Curve   [year]   (value)

---

## Average Sale Prices by Neighborhood

In [42]:
prices_by_year_by_neighborhood = sfo_data_df.groupby(["year","neighborhood"]).mean().drop(columns=["housing_units"])
prices_by_year_by_neighborhood.hvplot.line(groupby="neighborhood",x="year",
    xlabel="Year")

:DynamicMap   [neighborhood]
   :NdOverlay   [Variable]
      :Curve   [year]   (value)

---

#### Read Neighborhood Data

In [43]:
neighborhood_locations_df = pd.read_csv(
    Path("Resources/neighborhoods_coordinates.csv"),index_col="Neighborhood"
)        

#### Concatenate Price and Location Info

In [44]:
all_neighborhood_info_df = sfo_data_df.groupby("neighborhood").mean()

all_neighborhoods_df = pd.concat(
    [neighborhood_locations_df, all_neighborhood_info_df], 
    axis="columns",
    sort=False)

all_neighborhoods_df=all_neighborhoods_df.dropna()
all_neighborhoods_df["sale_price_sqr_foot"]=all_neighborhoods_df["sale_price_sqr_foot"].astype(float).round(2)
all_neighborhoods_df["gross_rent"]=all_neighborhoods_df["gross_rent"].astype(float).round(2)
all_neighborhoods_df["housing_units"]=all_neighborhoods_df["housing_units"].astype(int)
all_neighborhoods_df["year"]=all_neighborhoods_df["year"].astype(int)
all_neighborhoods_df = all_neighborhoods_df.reset_index().dropna()
all_neighborhoods_df = all_neighborhoods_df.rename(columns={"index": "Neighborhood"})

## Interactive Map w/ hvPlot + GeoViews

In [45]:
all_neighborhoods_df.hvplot.points("Lon","Lat",
                                   geo=True,
                                   tiles=True,
                                   size="sale_price_sqr_foot",
                                   frame_width=700,
                                   frame_height=500,
                                   color="gross_rent",
                                   hover_cols = ['Neighborhood'])

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lon,Lat]   (gross_rent,sale_price_sqr_foot,Neighborhood)